In [ ]:
import os
import sys
import math
import openpyxl
import utm
import ns.core
import ns.network
import ns.internet
import ns.applications
import ns.flow_monitor
import ns.wifi
import ns.mobility
import ns.aodv
app_path = os.getcwd() + "/app"
import kml_parser
import config

ns.core.Config.SetDefault("ns3::WifiRemoteStationManager::RtsCtsThreshold", ns.core.StringValue("0"))
ns.core.Config.SetDefault("ns3::WifiRemoteStationManager::FragmentationThreshold",ns.core.StringValue("2200"))

In [ ]:
def main():
    f = app_path + '/' + "topology_2000x100_50nodes.kml"
    topology_name, topology = kml_parser.get_nodes(f)
    print("\n=== Topology name: " + topology_name + " ===")
    aodv(topology)

In [ ]:
def aodv(topology):
    print "\n=== Exam AODV ==="
    convert_to_cartesian(topology)

    nodes = ns.network.NodeContainer()
    nodes.Create(len(topology))
    wifiChannel = ns.wifi.YansWifiChannelHelper()
    wifiChannel.SetPropagationDelay(config.propagation_delay_model)
    wifiChannel.AddPropagationLoss("ns3::LogDistancePropagationLossModel", "Exponent", ns.core.DoubleValue(3.0))
    wifiPhy = ns.wifi.YansWifiPhyHelper()
    wifiPhy.SetErrorRateModel(config.error_rate_model)
    wifiPhy.SetChannel(wifiChannel.Create())
    wifiMac = ns.wifi.WifiMacHelper()
    wifiMac.SetType("ns3::AdhocWifiMac")
    wifi = ns.wifi.WifiHelper()
    wifi.SetStandard(ns.wifi.WIFI_PHY_STANDARD_80211n_5GHZ)
    wifi.SetRemoteStationManager("ns3::ConstantRateWifiManager")
    stack = ns.internet.InternetStackHelper()
    stack.SetRoutingHelper(ns.aodv.AodvHelper())
    ipAddresses = ns.internet.Ipv4AddressHelper()
    ipAddresses.SetBase(ns.network.Ipv4Address("10.0.0.0"),
ns.network.Ipv4Mask("255.0.0.0"))
    stack.Install(nodes)

    wifiDevices = wifi.Install(wifiPhy, wifiMac, nodes)
    wifiInterfaces = ipAddresses.Assign(wifiDevices)
    positionList = ns.mobility.ListPositionAllocator()
    for i in range(len(topology)):
        positionList.Add(ns.core.Vector(topology[i].x,
topology[i].y, topology[i].z))
    mobility = ns.mobility.MobilityHelper()
    mobility.SetMobilityModel("ns3::ConstantPositionMobilityModel")
    mobility.SetPositionAllocator(positionList)
    mobility.SetMobilityModel("ns3::RandomWalk2dMobilityModel",
                              'Mode', ns.core.StringValue("Time"),
                              "Time", ns.core.StringValue("10s"),
                              "Speed",
ns.core.StringValue("ns3::ConstantRandomVariable[Constant=6.0]"),
                              "Bounds",
ns.mobility.RectangleValue(ns.mobility.Rectangle(0, 2000, 0, 200)))
    mobility.Install(nodes)
    for i in range(len(topology)):
        topology[i].ip = str(wifiInterfaces.GetAddress(i))
    flow_list = []
    for i, node in enumerate(topology):
        if node.dst:
            for dst_node in node.dst.keys():
                for flow in node.dst[dst_node]:
                    flow_params = []
flow = flow.split(":") 
flow_params.append(int(flow[0][1:])) 
flow_params.append(int(flow[1])) 
flow_params.append(int(flow[2][:-1])) 
flow_params.append(topology[i].ip)

flow_params.append(get_str_ipaddress_by_nodename(dst_node,topology))
                    flow_params.append(int(flow[3]))
                    flow_list.append(flow_params)
    for flow in flow_list:
        print get_nodename_by_str_ipaddress(str(flow[3]), topology), "=> [Class=" + str(flow[0]) + ', bitrate=' + str(flow[1]) + ', Flows=' + str(flow[2]), flow[3], \
            '-->', flow[4] + ':' + str(flow[5]) + "] =>",
get_nodename_by_str_ipaddress(str(flow[4]), topology)
    sink_apps_list = get_sink_apps(flow_list, topology, nodes)
    onoff_apps_list = get_onoff_apps(flow_list, topology, nodes)
    for app in sink_apps_list:
        app.Start(ns.core.Seconds(1))
        app.Stop(ns.core.Seconds(config.timer + config.duration))
    for app in onoff_apps_list:
        app.Start(ns.core.Seconds(config.timer))
        app.Stop(ns.core.Seconds(config.timer + config.duration))
flow_monitor
    monitorHelper = ns.flow_monitor.FlowMonitorHelper()
    monitor = monitorHelper.InstallAll()
    monitor = monitorHelper.GetMonitor()
    ns.core.Simulator.Stop(ns.core.Seconds(config.timer +
config.duration))
    ns.core.Simulator.Run()
    flow_dictionary = {}

150
    monitor.CheckForLostPackets()
    classifier = monitorHelper.GetClassifier()
    for flow_id, flow_stats in monitor.GetFlowStats():
        stat_dict = {}
        t = classifier.FindFlow(flow_id)
        if t.destinationPort > 1024:
            print "\nFlow_id: " + str(flow_id) + " " +
str(t.sourceAddress) + ":" + str(t.sourcePort), "-->", \
            str(t.destinationAddress) + ":" + str(t.destinationPort)
            flow_dictionary[flow_id] = get_statistics(flow_stats,
stat_dict, flow_list, topology, t)
            for item in flow_dictionary[flow_id]:
                print item + ":", flow_dictionary[flow_id][item]
    ns.core.Simulator.Destroy()
    return flow_dictionary
def get_statistics(flow_stats, stat_dict, flow_list, topology, t):
    stat_dict['class'] = get_class_by_flow(flow_list, t)
    stat_dict['src_node'] =
get_nodename_by_str_ipaddress(str(t.sourceAddress), topology)
    stat_dict['dst_node'] =
get_nodename_by_str_ipaddress(str(t.destinationAddress), topology)
    stat_dict['src_ip'] = t.sourceAddress
    stat_dict['src_port'] = t.sourcePort
    stat_dict['dst_ip'] = t.destinationAddress
    stat_dict['dst_port'] = t.destinationPort
    stat_dict['hop_cnt'] = int(flow_stats.timesForwarded) /
(flow_stats.rxPackets - 1) \
    if flow_stats.rxPackets != 1 else 0
    stat_dict['IPTD'] = flow_stats.delaySum.GetMilliSeconds() /
flow_stats.rxPackets \
    if flow_stats.rxPackets else False

151
    stat_dict['IPDV'] = flow_stats.jitterSum.GetMilliSeconds() /
(flow_stats.rxPackets - 1) \
    if flow_stats.rxPackets > 1 else False
    stat_dict['IPLR'] = float(flow_stats.lostPackets) /
float(flow_stats.rxPackets + flow_stats.lostPackets) \
    if flow_stats.lostPackets else 0
    for reason, cnt in enumerate(flow_stats.packetsDropped):
        packets_dropped[config.drop_reason[reason]] = cnt
    if 'DROP_BAD_CHECKSUM' in packets_dropped.keys():
        stat_dict['IPER'] =
float(packets_dropped['DROP_BAD_CHECKSUM']) /
float(flow_stats.rxPackets + flow_stats.lostPackets)
    else:
        stat_dict['IPER'] = 0
    stat_dict['original_bitrate'] = get_txbitrate_by_flow(flow_list,
t)
    if flow_stats.timeLastTxPacket.GetSeconds() !=
flow_stats.timeFirstTxPacket.GetSeconds():
        stat_dict['tx_bitrate'] = int(flow_stats.txBytes * 8 /
(flow_stats.timeLastTxPacket.GetSeconds() - \
        flow_stats.timeFirstTxPacket.GetSeconds()) / 1000)
    else:
        stat_dict['tx_bitrate'] = 0
    if flow_stats.timeLastRxPacket.GetSeconds() !=
flow_stats.timeFirstRxPacket.GetSeconds():
        stat_dict['rx_bitrate'] = int(flow_stats.rxBytes * 8 /
(flow_stats.timeLastRxPacket.GetSeconds() - \
        flow_stats.timeFirstRxPacket.GetSeconds()) / 1000)
    else:
        stat_dict['rx_bitrate'] = 0

152
    stat_dict['packets_dropped'] = packets_dropped
    return stat_dict
def get_class_by_flow(flow_list, t):
    for flow in flow_list:
        if flow[3] == str(t.sourceAddress) and flow[4] ==
str(t.destinationAddress) and flow[5] == t.destinationPort:
            return flow[0]
    print "Error in get_class_by_flow"
def get_txbitrate_by_flow(flow_list, t):
    for flow in flow_list:
        if flow[3] == str(t.sourceAddress) and flow[4] ==
str(t.destinationAddress) and flow[5] == t.destinationPort:
            return flow[1]
    print "Error in get_txbitrate_by_flow"
def get_onoff_apps(flow_list, topology, nodes):
    print "\n---OnOff---"
    onoff_apps_list = []
    for flow in flow_list:
        for num_flows in range(flow[2]):
            socketAddress =
ns.network.InetSocketAddress(ns.network.Ipv4Address(flow[4]),
flow[5])
            onoff =
ns.applications.OnOffHelper("ns3::UdpSocketFactory", socketAddress)
            bitrate = str(flow[1]) + 'kbps'
            onoff.SetConstantRate(ns.network.DataRate(bitrate),
config.packet_size)
topology)
node_index = get_node_index_by_str_ipaddress(flow[3],
onoffApp = onoff.Install(nodes.Get(node_index))
onoff_apps_list.append(onoffApp)

153
            node1 = nodes.Get(node_index)
            node2 =
nodes.Get(get_node_index_by_str_ipaddress(flow[4], topology))
            distance =
int(math.sqrt(ns.mobility.MobilityHelper.GetDistanceSquaredBetween(n
ode1, node2)))
            print "Installed on: " + topology[node_index].name + ":"
+ flow[3] + "-->" + flow[4] + ":" + str(flow[5]), \
            "Distance " + str(distance) + " Meters"
    return onoff_apps_list
def get_sink_apps(flow_list, topology, nodes):
    sockets = {}
    for flow in flow_list:
        if flow[4] not in sockets:
            sockets[flow[4]] = []
        if flow[5] not in sockets[flow[4]]:
            sockets[flow[4]].append(flow[5])
        if not len(sockets):
            "No servers for packetsink in topology"
            exit()
    print "\n---PacketSink---"
    sink_apps_list = []
    for ip in sockets:
        node_index = get_node_index_by_str_ipaddress(ip, topology)
        for port in sockets[ip]:
            socketAddress =
ns.network.InetSocketAddress(ns.network.Ipv4Address(ip), port)
            packetSink =
ns.applications.PacketSinkHelper("ns3::UdpSocketFactory",
socketAddress)
            sinkApp = packetSink.Install(nodes.Get(node_index))
            sink_apps_list.append(sinkApp)

154
            print "Installed on: " + topology[node_index].name + ":"
+ ip + ":" + str(port)
    return sink_apps_list
def get_node_index_by_str_ipaddress(ipaddress, topology):
    for i, node in enumerate(topology):
        if node.ip == ipaddress:
            return i
    print "Error in get_node_index_by_ipaddress"
    exit()
def get_nodename_by_str_ipaddress(ipaddress, topology):
    for node in topology:
        if node.ip == ipaddress:
            return node.name
    print "Error in get_nodename_by_ipaddress"
    exit()
def get_str_ipaddress_by_nodename(nodename, topology):
    for node in topology:
        if node.name == nodename:
            return node.ip
    print "Error in get_str_ipaddress_by_nodename"
    exit()
def get_ipaddress_by_nodename(nodename, topology, wifiInterfaces):
    for i in range(len(topology)):
        if topology[i].name == nodename:
            return wifiInterfaces.GetAddress(i)
    print "Error in get_ip_address_by_nodename"
exit()
def convert_to_cartesian(topology):
    for i, node in enumerate(topology):
        xy = utm.from_latlon(node.lat, node.lon)

155
        topology[i].x = int(xy[0])
        topology[i].y = int(xy[1])
    min_x = topology[0].x
    min_y = topology[0].y
    for node in topology:
        min_x = node.x if node.x < min_x else min_x
        min_y = node.y if node.y < min_y else min_y
    for node in topology:
        node.x -= min_x
        node.y -= min_y
        print(node.name, node.x, node.y)
if __name__ == "__main__":
    main()
